In [1]:
import time
import math
import tensorflow as tf
import keras 
import numpy as np
import pandas as pd
from keras.models import load_model,Model
from keras.engine.topology import Layer

C:\Users\ad4\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ad4\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ad4\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ad4\AppData\Loca

In [2]:
# 定义融合层，将深度学习算法与历史均值算法融合
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [3]:
#定义精度评价指标。为防止0值附近相对误差过大而导致的异常，定义mask层。
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [4]:
time3 = time.time()

distance = np.asarray(pd.read_csv('target_dis.csv',header = 0, index_col= 0))
near_road = np.argsort(distance)

flow = np.array(pd.read_csv(r"speed_target.csv", header= 0, index_col= 0)) #注意header=0 or None

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 31 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 82 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(144-t_input-t_pre):
        image3.append(image2[:, i*144+j:i*144+j+t_input, :][:])
        label.append(flow[:, i*144+j+t_input:i*144+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*144+j+t_input:(i-1)*144+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_target = image3[:np.shape(image3)[0]*1//31]
image_test_target = image3[np.shape(image3)[0]*1//31:]
label_train_target = label[:np.shape(label)[0]*1//31]
label_test_target = label[np.shape(label)[0]*1//31:]

day_train_target = day[:np.shape(day)[0]*1//31]
day_test_target = day[np.shape(day)[0]*1//31:]


time4 = time.time()
print('input done %g' % (time4-time3))


(3870, 5, 12, 82)
(3870, 82, 3)
(3870, 82, 3)
input done 0.0488718


In [5]:
time3 = time.time()

distance = np.asarray(pd.read_csv('source_dis.csv',header = 0, index_col= 0))
near_road = np.argsort(distance)

flow = np.array(pd.read_csv(r"speed_source.csv", header= 0, index_col= 0)) #注意header=0 or None

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 31 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 82 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(144-t_input-t_pre):
        image3.append(image2[:, i*144+j:i*144+j+t_input, :][:])
        label.append(flow[:, i*144+j+t_input:i*144+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*144+j+t_input:(i-1)*144+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_source = image3[:np.shape(image3)[0]*1//31]
image_test_source = image3[np.shape(image3)[0]*1//31:]
label_train_source = label[:np.shape(label)[0]*1//31]
label_test_source = label[np.shape(label)[0]*1//31:]

day_train_source = day[:np.shape(day)[0]*1//31]
day_test_source = day[np.shape(day)[0]*1//31:]



time4 = time.time()
print('input done %g' % (time4-time3))

(3870, 5, 12, 82)
(3870, 82, 3)
(3870, 82, 3)
input done 0.0488687


In [6]:
#模型构建
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = 30,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = 30,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 82)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 82)    328         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    22170       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 30)    0           conv2d_1[0][0]       

In [7]:
#参数加载
finish_model.load_weights('source1.h5')


In [8]:
#模型预测
model_pre = finish_model.predict([image_test_target,day_test_target])


In [9]:
#预测结果存储
# model_pre = np.reshape(model_pre,[103, 6])
# model_pre1 = pd.DataFrame(model_pre)
# model_pre1.to_csv('预测值.csv', index = False)

In [10]:
#transfer without FT 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_target)
smape_mean = smape_loss_func(model_pre, label_test_target)
mae_mean = mae_loss_func(model_pre, label_test_target)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.8883346348539743
smape = 1.1060631285712377
mae = 50.212213331057384


In [11]:
from keras import backend as K
from sklearn import metrics
def mmd (x, y):
    return metrics.mean_squared_error(x,y)

def new_loss(output_final, label_train_target):
    middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)
    middle_result_source = middle.predict([image_train_source, day_train_source])
    middle_result_target = middle.predict([image_train_target, day_train_target])

    loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
    loss2 = 0.01 * mmd (middle_result_source, middle_result_target)
    overall_loss = loss1 + loss2
    return overall_loss


In [12]:
finish_model.compile(optimizer='adam',loss=new_loss)

In [13]:
finish_model.fit([image_train_target, day_train_target], label_train_target, epochs=600, batch_size=512,
validation_data=([image_test_target,day_test_target], label_test_target), verbose = 2)

Train on 124 samples, validate on 3746 samples
Epoch 1/600
 - 1s - loss: 1147.8462 - val_loss: 1500.1749
Epoch 2/600
 - 0s - loss: 1045.4607 - val_loss: 1461.1951
Epoch 3/600
 - 0s - loss: 1018.6842 - val_loss: 1449.2982
Epoch 4/600
 - 0s - loss: 995.3873 - val_loss: 1448.3197
Epoch 5/600
 - 0s - loss: 971.0936 - val_loss: 1438.2459
Epoch 6/600
 - 0s - loss: 955.2532 - val_loss: 1426.5414
Epoch 7/600
 - 0s - loss: 943.3039 - val_loss: 1419.5918
Epoch 8/600
 - 0s - loss: 934.0153 - val_loss: 1414.5475
Epoch 9/600
 - 0s - loss: 925.4224 - val_loss: 1414.3330
Epoch 10/600
 - 0s - loss: 914.7467 - val_loss: 1418.8998
Epoch 11/600
 - 0s - loss: 905.8820 - val_loss: 1418.5711
Epoch 12/600
 - 0s - loss: 895.9669 - val_loss: 1401.5500
Epoch 13/600
 - 0s - loss: 892.4910 - val_loss: 1374.4628
Epoch 14/600
 - 0s - loss: 883.0661 - val_loss: 1339.0083
Epoch 15/600
 - 0s - loss: 875.9177 - val_loss: 1309.5152
Epoch 16/600
 - 0s - loss: 870.0261 - val_loss: 1288.4122
Epoch 17/600
 - 0s - loss: 864.

In [14]:
model_pre = finish_model.predict([image_test_target,day_test_target])

In [15]:
#transfer with DAN 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_target)
smape_mean = smape_loss_func(model_pre, label_test_target)
mae_mean = mae_loss_func(model_pre, label_test_target)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.18426038573568623
smape = 0.17835187066308672
mae = 9.409976378659644


In [16]:
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test_target[:,i,:])
    mape_list.append(a1)

mape_pd = pd.Series(mape_list)
mape_pd.sort_values()

71    0.048404
77    0.074991
45    0.084538
49    0.085888
64    0.086190
        ...   
3     0.311950
11    0.318855
69    0.413262
20    0.449381
17    0.897117
Length: 82, dtype: float64